# SARAH Toolbox

## Collection of models related to the 750 GeV diphoton excess

See for more details [1602.05581](http://arxiv.org/abs/1602.05581)

[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/BSM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)


## Implemented models

* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/diphotonModels/LRmodels/darkLR/`
* `SARAH/Models/diphotonModels/LRmodels/tripletLR/`  (with additional Z_2 and T_i dark matter)

## Test
Below we check the SSDM models. It is recomended that the commands be executed [from a terminal](../terminals/1).

Be sure that `SPHENO/Makefile` have `gfortan` uncommented:

In [1]:
cat  SPHENO/Makefile | grep "F90 ="

# F90 = nagfor
F90 = gfortran
# F90 = g95
# F90 = lf95
# F90 = ifort


In [2]:
%%bash
./compile_spheno_directly.sh SSDM

make: Entering directory '/home/main/notebooks/SPHENO'
cd /home/main/notebooks/SPHENO/SSDM ; make F90=gfortran version=330.00
make[1]: Entering directory '/home/main/notebooks/SPHENO/SSDM'
cd ../src ; make F90=gfortran 
make[2]: Entering directory '/home/main/notebooks/SPHENO/src'
gfortran -c -O -J../include -I../include  -DGENERATIONMIXING  -DONLYDOUBLE Control.F90
ar -ruc ../lib/libSPheno.a Control.o
rm -f Control.o
gfortran -c -O -J../include -I../include Mathematics.f90
ar -ruc ../lib/libSPheno.a Mathematics.o
rm -f Mathematics.o
gfortran -c -O -J../include -I../include  -DGENERATIONMIXING  -DONLYDOUBLE RGEs.F90
ar -ruc ../lib/libSPheno.a RGEs.o
rm -f RGEs.o
gfortran -c -O -J../include -I../include MathematicsQP.f90
ar -ruc ../lib/libSPheno.a MathematicsQP.o
rm -f MathematicsQP.o
gfortran -c -O -J../include -I../include LoopFunctions.f90
ar -ruc ../lib/libSPheno.a LoopFunctions.o
rm -f LoopFunctions.o
gfortran -c -O -J../include -I../include StandardModel.f90
ar -ruc ../lib/libSPhe

RGEs.F90:98:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
RGEs.F90:14740:0: warning: extra tokens at end of #endif directive
 #endif SEESAWIII
 ^
RGEs.F90:19645:0: warning: extra tokens at end of #endif directive
 #endif SEESAWIII
 ^
SugraRuns.F90:1886:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:2783:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:2905:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:3099:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:4225:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:5118:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:5180:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
SugraRuns.F90:6101:0: warning: extra tokens at end of #endif directive
 # endif SEESAWI

make a work dir

In [3]:
mkdir work

In [4]:
cd work

/home/main/notebooks/work


If neccesary, turn ON tachyonic spectrum with `52 1 #...`

In [5]:
cat ../SPHENO/SSDM/LesHouches.in.SSDM | grep 'Write spectrum in case of tachyonic states' 

 52 0               # Write spectrum in case of tachyonic states 


In [6]:
%%bash
../SPHENO/bin/SPhenoSSDM ../SPHENO/SSDM/LesHouches.in.SSDM

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


In [7]:
%%bash
cd ../micromegas
make
make
cd SSDM
make main=CalcOmega.cpp

echo \#define micrO \"/home/main/notebooks/micromegas\"  > sources/microPath.h
make -C CalcHEP_src
make[1]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src'
./getFlags
MAKE= make
UNIX Linux
C compiler detected
can not generate shared library
tring to add -fPIC option
: shared liblary generation OK
Trying to detect X11
search by xmkmf
      fails
search X11 according to list of possible distinations
      fails
X11 not detected. CalcHEP will be compiled in Blind mode
Fortran compiler is detected
gfortran OK
FFLAGS OK
./sbin/setPath /home/main/notebooks/micromegas/CalcHEP_src
chmod 755 mkWORKdir
make -C c_source
make[2]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src/c_source'
make -C getmem 
make[3]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src/c_source/getmem'
gcc -g -fsigned-char -Wstrict-prototypes -Wimplicit -Wmissing-prototypes -std=gnu99 -fPIC -Iinclude -I../chep_crt/include   -c -o getmem.o getmem.c
ar rv ../../lib/serv.a getmem

ar: creating ../../lib/serv.a
ar: creating ../../lib/symb.a
ar: creating /home/main/notebooks/micromegas/CalcHEP_src/lib/ntools.a
vegas.c:48:6: warning: no previous prototype for 'setEventCubes' [-Wmissing-prototypes]
 void setEventCubes(vegasGrid*vegPtr, long nCubes)
      ^
ar: creating ../../lib/dynamic_me.a
ar: creating ../../lib/dynamic_vp.a
ar: creating /home/main/notebooks/micromegas/CalcHEP_src/lib/num_c.a
runVegas.c: In function 'readVegasGrid':
runVegas.c:346:5: warning: implicit declaration of function 'setEventCubes' [-Wimplicit-function-declaration]
     setEventCubes(veg_Ptr, nCubes);
     ^
ar: creating /home/main/notebooks/micromegas/CalcHEP_src/lib/dummy.a
pdf_dummy.c:7:9: warning: no previous prototype for 'alphaspdfm_' [-Wmissing-prototypes]
 double  alphaspdfm_(int*LHA, double *Q ){ return 0.12;}
         ^
dummy_output.c:43:8: warning: no previous prototype for 'varAddress' [-Wmissing-prototypes]
 REAL * varAddress(char *name) { return 0;}
        ^
dummy_output.c:

Run micromegas

In [8]:
%%bash
../micromegas/SSDM/CalcOmega SPheno.spc.SSDM

PROCESS: ~Ss,~Ss->AllEven,1*x{,h,g,A,Z,Wp,Wm,nu1,Nu1,nu2,Nu2,nu3,Nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3


mv: cannot move 'omg_p1p1_u.so' to '/home/main/notebooks/micromegas/SSDM/work/so_generated/omg_p1p1_u.so': No such file or directory
bash: line 1:  1490 Segmentation fault      ../micromegas/SSDM/CalcOmega SPheno.spc.SSDM
